In [1]:
from nptdms import TdmsFile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from array import array
np.seterr(all='ignore')
from enum import Enum
import json
import math
import datetime

In [2]:
def Read_Groups_and_Channels(tdms_file):
    # Loop through each group and print the channel names
    for group in tdms_file.groups():
        print(f"Group '{group.name}':")
        for channel in group.channels():
            print(f"- Channel '{channel.name}':")

In [ ]:
DEBUG = False
DISPLAY = False
########## 20240223 Prima data ##########
NpulsePerTrigger=1
# Define signal region
Pulse_startT     =  308 #312 #67 #200 #215
Pulse_endT       =  380 #125 #215 #245
# Define rising and falling separation
Pulse_rise_endT     =  313 #75 #200 #215
Pulse_fall_endT     =  342 #75 #200 #215
# Define pre-pulse (sideband) region
prePulse_startT  =  200 #100
prePulse_endT    =  300 #160
# Define post-pulse (sideband) region
postPulse_startT =  600 #230
postPulse_endT   =  800 #250
# Sideband pre-selection
cut_preRange = 0.11
cut_posRange = 10
cut_preStd = 0.021
cut_posStd = 10
# Pulse selection
cut_pulseRange = 0.05
# FFT
freq_steps = 10000
totalTreeEvents = 10000
avgMaxCount = 3000

In [ ]:
if (args.debug_report==True): cf.DEBUG = True
if (args.display_report==True): cf.DISPLAY = True
ROOT.gStyle.SetPalette(ROOT.kBird)

########## Init ##########
for in_filename in args.in_filenames:
print("\n##############################")
print(f"input file: {in_filename}")
basename = in_filename.rsplit('/',1)[1].split('.tdms')[0]
baseDir = in_filename.split('SNSPD_rawdata/')[1].rsplit('/',1)[0]
outDir = args.outputDir + '/' + baseDir + '/' + basename
metaFileName = outDir + '/' + in_filename.rsplit('/',1)[1].split('.tdms')[0] + ".json"
createDir(outDir)
# Create root filen
outfile = ROOT.TFile(f'{outDir}/{basename}.root', 'RECREATE', f'analysis histograms of {basename} measurements' )
outtree = ROOT.TTree("Result_tree","Pulse laser analysis results")
out_inputname = ROOT.TNamed("inputDataName",in_filename)
out_metaname = ROOT.TNamed("metaDataName",metaFileName)
# Define variables for branch
pre_std, pos_std, pre_mean, pos_mean, pre_range, pos_range, pre_max, pos_max = array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0])
pulse_rise_range, pulse_fall_range, pulse_rise_range_ptb, pulse_fall_range_ptp = array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0])
pulse_amplitude, pulse_arrivalT, pulse_riseT, pulse_pre_range = array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0])
pulse_FWHM = array('f',[0])
pulse_fall_tau, pulse_fall_A, pulse_fall_t0, pulse_fall_C, pulse_fall_status = array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0])
pulse_rise_tau = array('f',[0])
pulse_max, pulse_min, pulse_max_T, pulse_min_T = array('f',[0]),array('f',[0]),array('f',[0]),array('f',[0])
# init branches
outtree.Branch('pre_std', pre_std, 'pre_std/F')
outtree.Branch('pre_mean', pre_mean, 'pre_mean/F')
outtree.Branch('pre_range', pre_range, 'pre_range/F')
outtree.Branch('pre_max', pre_max, 'pre_max/F')
outtree.Branch('pos_std', pos_std, 'pos_std/F')
outtree.Branch('pos_mean', pos_mean, 'pos_mean/F')
outtree.Branch('pos_range', pos_range, 'pos_range/F')
outtree.Branch('pos_max', pos_max, 'pos_max/F')
outtree.Branch('pulse_rise_range', pulse_rise_range, 'pulse_rise_range/F')
outtree.Branch('pulse_rise_range_ptb', pulse_rise_range_ptb, 'pulse_rise_range_ptb/F')
outtree.Branch('pulse_fall_range', pulse_fall_range, 'pulse_fall_range/F')
outtree.Branch('pulse_fall_range_ptp', pulse_fall_range_ptp, 'pulse_fall_range_ptp/F')
outtree.Branch('pulse_max', pulse_max, 'pulse_max/F')
outtree.Branch('pulse_min', pulse_min, 'pulse_min/F')
outtree.Branch('pulse_max_T', pulse_max_T, 'pulse_max_T/F')
outtree.Branch('pulse_min_T', pulse_min_T, 'pulse_min_T/F')
outtree.Branch('pulse_rise_tau', pulse_rise_tau, 'pulse_rise_tau/F')
outtree.Branch('pulse_fall_tau', pulse_fall_tau, 'pulse_fall_tau/F')
outtree.Branch('pulse_fall_A', pulse_fall_A, 'pulse_fall_A/F')
outtree.Branch('pulse_fall_t0', pulse_fall_t0, 'pulse_fall_t0/F')
outtree.Branch('pulse_fall_C', pulse_fall_C, 'pulse_fall_C/F')
outtree.Branch('pulse_fall_status', pulse_fall_status, 'pulse_fall_status/F')
outtree.Branch('pulse_FWHM', pulse_FWHM, 'pulse_FWHM/F')
# outtree.Branch('pulse_pre_range', pulse_pre_range, 'pulse_pre_range/F')
if (args.doAdvanced):
    outtree.Branch('pulse_amplitude', pulse_amplitude, 'pulse_amplitude/F')
    outtree.Branch('pulse_arrivalT', pulse_arrivalT, 'pulse_arrivalT/F')
    outtree.Branch('pulse_riseT', pulse_riseT, 'pulse_riseT/F')
########## End Init ##########

# Start Analysis
SingleTDMS_analysis()
# plots()
# End Analysis
print (f'Output: {outDir}/{basename}.root')
out_inputname.Write()
out_metaname.Write()
outtree.Write()
outfile.Close()
